In [ ]:
# implementation of the gentic algorithm

In [ ]:
# imports
import numpy as np

In [ ]:
class GeneticAlgorithm:
    
    def __init__(self, pop_size = 100, n_iter = 10, fitness_model = None, vocab = None, cross_rate = 0.8, mut_rate = 0.2, ind_shape = (70, 100), seed = 1234, verbose = 1):
        '''
        Genetic algorithm implementation using numpy
        '''
        self.pop_size      = pop_size
        self.n_iter        = n_iter
        self.fitness_model = fitness_model
        self.vocab         = vocab
        self.cross_rate    = cross_rate
        self.mut_rate      = mut_rate
        self.ind_shape     = ind_shape
        self.seed          = seed
        self.verbose       = verbose

        # prints only if verbose > 0
        if self.verbose > 0:
            print("Genetic algorithm with:")
            print("\t- population size: {}\n\t- crossover rate: {}\n\t- mutation rate: {}\n".format(self.pop_size, self.cross_rate, self.mut_rate))

    def start_algorithm(self):
        '''
        Fucntion that executes the algorithm 
        '''
        
        # fix the seed to make it deterministic
        np.random.seed(self.seed)

        # generate the initial population
        population = self.generate_population()

        # evaluate it
        scores = self.evaluate_population(population)
        stopCondition = False
        
        # main iteration loop
        for i in range(self.n_iter):
            
            # perform the selection of the population accordint to their fitness
            sel_population = self.select_population(population, scores)
            # perform the crossover of the selected population
            cros_population = self.crossover_population(sel_population)
            # mutate the crossover population
            mut_population = self.mutation_population(cros_population)
            # evaluate it
            mut_scores = self.evaluate_population(mut_population)
            # combine the new population with the previous one
            population, scores = self.combine_population(population, mut_population, scores, mut_scores)
            
            # print iteration information
            if self.verbose > 0:
                print("generation {}\n- avg fitness: {}\n- max fitness: {}".format(i, np.mean(scores), np.max(scores)))
            
    
    def generate_population(self):
        '''
        Function that generates a population of a given size
        with the desired representation
        '''
        
        # generate a random population using the available words in w2v
        population_words = np.random.choice(list(self.vocab.keys()), (self.pop_size, self.ind_shape[0]))
        population = np.array([[{iw:self.vocab[iw][0]} for iw in ind_words] for ind_words in population_words])

        return population
    
    def evaluate_population(self, pop):
        '''
        Function that returns the fitness of the elements of
        a given population
        '''

        # obtain the scores of the individuals 
        #scores = self.fitness_model.predict(pop)
        scores = np.random.rand(self.pop_size)
        
        return scores
    
    def select_population(self, pop, scores, n_tournament = 2):
        '''
        Function that performs tournament selection 
        '''
        
        # list containing the selected individuals
        sel_individuals = []
        
        # select pop_size individuals
        for i in range(len(pop)):
            
            # obtain n_tournament random indices
            indxs = np.random.randint(0, len(pop), n_tournament)
            # get the index of the individual with the maximum fitness
            #print(scores[indxs])
            a = np.argmax(scores[indxs])
            sel_individual_indx = indxs[a]
            # append the individual to the selection list
            sel_individuals.append(pop[sel_individual_indx])
        
        return np.array(sel_individuals)
    
    def crossover_population(self, pop):
        '''
        Function that performs crossover over a given population
        '''
        
        # iterate over all individuals in the population
        for i in range(0, len(pop), 2):
            # decie if crossover or not over this pair of 
            if np.random.rand() > self.cross_rate:
                # pick a random point in the configuration
                randinx = np.random.randint(70)
                aux_conf = pop[i,:randinx,:]
                pop[i,:randinx] = pop[i+1,:randinx,:]
                pop[i+1,:randinx] = aux_conf
        
        return pop
                        
    def mutation_population(self, pop):
        '''
        Function that performs mutation over a given population
        '''
        
        # iterate over all individuals in the population
        for individual in pop:
            # iterate over the configurations in the individual
            for configuration in individual:
                # decide if mutate or not this given configuration
                if np.random.rand() > self.mut_rate:
                    configuration = np.random.rand(100)
        
        return pop
        
    def combine_population(self, population, mut_population, scores, mut_scores):
        '''
        Function that combines the old and new population into a new one
        '''
        
        # variables that will hold the new population and its corresponding scores
        new_population = np.empty(population.shape)
        new_scores = np.empty(scores.shape)
        
        # select the best 30% of the old individuals and the 70% of the new oens
        n_olds = int(len(scores) * 0.3)
        n_news = len(scores) - n_olds
        indxs_old = np.argsort(scores)[-n_olds:][::-1]
        indxs_new = np.argsort(mut_scores)[-n_news:][::-1]
        
        # assign the new population and scores
        new_population[:n_olds] = population[indxs_old]
        new_scores[:n_olds] = scores[indxs_old]
        new_population[n_olds:] = mut_population[indxs_new]
        new_scores[n_olds:] = mut_scores[indxs_new]
        
        return new_population, new_scores

In [ ]:
a = GeneticAlgorithm()

In [ ]:
a.start_algorithm()